In [1]:
import requests
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import time

In [2]:
#!pip install markdown

In [3]:
from code import interact
import pandas as pd
import markdown
from requests import options

In [4]:
def fetch_event_ids(game_id, game):   
    all_plays = pd.DataFrame(game['liveData']['plays']['allPlays'])
    event_ids = pd.json_normalize(all_plays['about'])['eventId'].values
    coordinates = all_plays.coordinates.values
    results = all_plays.result.values
    event_cord = dict(zip(event_ids,coordinates))
    return event_cord 
    
def get_game_ids(path):
    """returns a list of all game ids inside a json file"""
    with open(path) as json_file:
        data = json.load(json_file)
    game_ids = list(data.keys())
    return game_ids

def game_id_widget(path):
    """"returns Game ID widget """
    
    game_ids = get_game_ids(path)
      
    g_id_widget = widgets.SelectionSlider(
        value=min(game_ids),
        min=max(game_ids),
        max=max(game_ids),
        options = game_ids,
        description='game_id:',
        continuous_update=True
    )
    
    return g_id_widget


def event_id_widget(event_ids):
    
    e_id_widget = widgets.SelectionSlider(
        min=min(event_ids),
        max=max(event_ids),
        options = event_ids,
        description='event_id:',
        continuous_update=False
    )
    return e_id_widget


def plot_image():
    image = plt.imread("./references/nhl_rink.png")
    fig, ax = plt.subplots(figsize=(9,4))
    ax.set_xlabel('Feet', fontweight ='bold')
    ax.set_ylabel('Feet', fontweight ='bold')
    ax.set_title("Result Description", fontsize = 10, fontweight ='bold')
    ax.imshow(image, extent=[-100, 100, 42.5, -42.5])
    plt.plot(20,20, "ob", markersize=8)
    plt.show()


def display_details(**args):
    """displays the event start date and time according to selected game_id"""
    with open(args["path"]) as json_file:
        data = json.load(json_file)

    game = data[str(args["game_id"])]
    all_plays = pd.DataFrame(game['liveData']['plays']['allPlays'])
    event_ids = pd.json_normalize(all_plays['about'])['eventId'].values
    
    coordinates = all_plays.coordinates.values
    results = all_plays.result.values
    event_cord = dict(zip(event_ids,coordinates))
    
    
    display(args["game_id"])
    
    a = "Game Start Date: " + game['gameData']['datetime']['dateTime']
    display(a) 
    
    b = "Game ID: " + str(args["game_id"])+"; "+game['gameData']['teams']['home']['abbreviation']+" (home) vs "+game['gameData']['teams']['away']['abbreviation']+" (away)"
    # Display the 2 teams information in a form of a dataframe
    display(b)
    
    df = pd.DataFrame({'Team': [game['gameData']['teams']['home']['abbreviation'], game['gameData']['teams']['away']['abbreviation']],
                        'Goals': [game['liveData']['linescore']['teams']['home']['goals'], game['liveData']['linescore']['teams']['away']['goals']],
                        'SOG': [game['liveData']['linescore']['teams']['home']['shotsOnGoal'], game['liveData']['linescore']['teams']['away']['shotsOnGoal']],
                        'SO Goals': [game['liveData']['linescore']['teams']['home']['powerPlay'], game['liveData']['linescore']['teams']['away']['powerPlay']],
                        #'SO Attempts': [game['liveData']['linescore']['teams']['home']['powerPlayOpportunities'], game['liveData']['linescore']['teams']['away']['powerPlayOpportunities']]
                        })  
    display(df)
    
    event_ids = event_id_widget(event_ids)
    display(event_ids)
    
    current_event_id = event_ids.value
    display(current_event_id)
    
    display(plot_image())
    
    all_plays = pd.DataFrame(game['liveData']['plays']['allPlays'])
    about = pd.json_normalize(all_plays['about'])
    specific_event_about = {"about": about.loc[about['eventId'] == current_event_id].reset_index(drop=True).to_dict()}
    event_ids = pd.json_normalize(all_plays['about'])['eventId'].values
    coordinates = all_plays.coordinates.values
    players = all_plays.players.values
    team = all_plays.team.values
    result = all_plays.result.values
    
    con_data = pd.DataFrame(list(zip(event_ids,coordinates,players,team,result)), columns=['event_ids','coordinates','players','team','result'])
    
    specific_event = con_data.loc[about['eventId'] == current_event_id].reset_index(drop=True).to_dict()
    final_dict = {**specific_event_about, **specific_event}
    display(final_dict)
    

In [5]:
widgets.interact(display_details, game_id=game_id_widget('./data/2016_playoffs.json') , path=widgets.fixed('./data/2016_playoffs.json'))

FileNotFoundError: [Errno 2] No such file or directory: './data/2016_playoffs.json'